In [2]:
import sys
sys.path.append("..")

from core.data_manager import retrieve_data_offline 
from core.constants import BEAR_PATTERNS, BULL_PATTERNS, BEAR_PATTERNS2, BULL_PATTERNS2
from core import indicators as ic
from core import macro
from core.common import remove_multi
import vectorbtpro as vbt
import gc
import numbers

import pandas as pd
import numpy as np
from numba import njit
import joblib

from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

import pickle

from core import strat, stratL, constants, common, presel
import math

from keras import Sequential,Input
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

2023-10-19 18:12:22.684337: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-19 18:12:22.684362: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
class ML():
    def __init__(
            self,
            period: str,
            indexes: list=["CAC40", "DAX", "NASDAQ"], #,"NYSE",,"FIN","HEALTHCARE","IT"
            ):

        #init
        for k in ["indexes"]:
            setattr(self,k,locals()[k])
        
        for key in ["close","open","low","high","data","volume"]:
            setattr(self,key+"_dic",{})
            setattr(self,key+"_ind_dic",{})

        for ind in self.indexes:
            retrieve_data_offline(self,ind,period)                
            self.data_dic[ind]=self.data
            for d in ["Close","Open","Low","High","Volume"]:
                getattr(self,d.lower()+"_dic")[ind]=self.data_dic[ind].get(d)   
                getattr(self,d.lower()+"_ind_dic")[ind]=self.data_ind.get(d)   
            
    def prepare(self,
               test_size:numbers.Number=0.2,
               data_name:str=None,
               preprocessing:bool=True,
               next_day_price:bool=True,
               distance:int=30,
               lag:int=0
               ):
        
        if data_name is None:
            self.defi_x(preprocessing=preprocessing, lag=lag)
            self.x_df, self.x_train, self.x_test=self.flatten(self.all_x)
            self.defi_y(next_day_price=next_day_price, distance=distance)
            self.y_df, self.y_train, self.y_test=self.flatten(self.all_y)
        else:
            self.x_df=pd.read_csv("x_"+data_name+".csv",index_col=[0,1,2],parse_dates=True)
            self.y_df=pd.read_csv("y_"+data_name+".csv",index_col=[0,1,2],parse_dates=True)
            self.x_train=pd.read_csv("x_train_"+data_name+".csv",index_col=[0,1,2],parse_dates=True)
            self.y_train=pd.read_csv("y_train_"+data_name+".csv",index_col=[0,1,2],parse_dates=True)
            self.x_test=pd.read_csv("x_test_"+data_name+".csv",index_col=[0,1,2],parse_dates=True)
            self.y_test=pd.read_csv("y_test_"+data_name+".csv",index_col=[0,1,2],parse_dates=True)
        print("preparation finished")
        
    def save(self,data_name:str):
        self.x_df.to_csv("x_"+data_name+".csv")
        self.y_df.to_csv("y_"+data_name+".csv")
        self.x_train.to_csv("x_train_"+data_name+".csv")
        self.y_train.to_csv("y_train_"+data_name+".csv")        
        self.x_test.to_csv("x_test_"+data_name+".csv")
        self.y_test.to_csv("y_test_"+data_name+".csv")
        
    def defi_x(
        self,
        preprocessing:bool=True,
        sample_size:int=50,
        lag:int=0
    ):
        self.all_x={}
        for ind in self.indexes: #CAC, DAX, NASDAQ
            all_x={}
            open_=self.open_dic[ind].shift(1)
            high=self.high_dic[ind].shift(1)
            low=self.low_dic[ind].shift(1)
            close=self.close_dic[ind].shift(1)
            close_ind=self.close_ind_dic[ind].shift(1)
            volume=self.volume_dic[ind].shift(1)
            
            if preprocessing:
                #Use preprocessed signals as input
                
                t=ic.VBTSTOCHKAMA.run(high,low,close)
                #all_x['STOCH_ent']=t.entries_stoch.astype(float)
                #all_x['STOCH_ex']=t.exits_stoch.astype(float)
                all_x['STOCH']=t.stoch

                #all_x['KAMA_ent']=t.entries_kama.astype(float)
                #all_x['KAMA_ex']=t.exits_kama.astype(float)

                #t=ic.VBTSUPERTREND.run(high,low,close)
                #all_x['SUPERTREND_ent']=t.entries.astype(float)
                #all_x['SUPERTREND_ex']=t.exits.astype(float)

                #t=vbt.BBANDS.run(close)
                #all_x['BBANDS_ent']=t.lower_above(close).astype(float)
                #all_x['BBANDS_ex']=t.upper_below(close).astype(float)
                #all_x["bandwidth"]=t.bandwidth
                #all_x["bandwidth_above"]=t.bandwidth_above(close)
                #all_x["bandwidth_below"]=t.bandwidth_below(close)

                t=vbt.RSI.run(close,wtype='simple')
                all_x['RSI']=t.rsi

                #all_x['RI20_ent']=t.rsi_crossed_below(20).astype(float)
                #all_x['RI20_ex']=t.rsi_crossed_above(80).astype(float)

                #all_x['RI30_ent']=t.rsi_crossed_below(30).astype(float)
                #all_x['RI30_ex']=t.rsi_crossed_above(70).astype(float)
                
                t=ic.VBTWILLR.run(high,low, close)
                all_x["WILLR"]=t.out
                
                t=vbt.talib("ULTOSC").run(high, low, close)
                all_x['ULTOSC']=t.real
                
                t=vbt.talib("MFI").run(high, low, close, volume)
                all_x['MFI']=t.real
                
                for func_name in BULL_PATTERNS2:
                    all_x[func_name+'_ent']=ic.VBTPATTERNONE.run(open_,high,low,close,func_name, "ent",True).out.astype(float)

                for func_name in BEAR_PATTERNS2:
                    all_x[func_name+'ex']=ic.VBTPATTERNONE.run(open_,high,low,close,func_name, "ex",True).out.astype(float)

                all_x["GROW_30"]=ic.VBTGROW.run(close,distance=30, ma=False).out
                all_x["GROW_30_rank"]=all_x["GROW_30"].rank(axis=1, ascending=False)
                all_x["GROW_30_ma"]=ic.VBTGROW.run(close,distance=30, ma=True).out
                all_x["GROW_30_ma_rank"]=all_x["GROW_30_ma"].rank(axis=1, ascending=False)
                all_x["GROW_30_dema"]=ic.VBTGROW.run(close,distance=30, dema=True).out
                all_x["GROW_30_dema_rank"]=all_x["GROW_30_dema"].rank(axis=1, ascending=False)

                all_x["GROW_50"]=ic.VBTGROW.run(close,distance=50, ma=False).out
                all_x["GROW_50_rank"]=all_x["GROW_50"].rank(axis=1, ascending=False)
                all_x["GROW_50_ma"]=ic.VBTGROW.run(close,distance=50, ma=True).out
                all_x["GROW_50_ma_rank"]=all_x["GROW_50_ma"].rank(axis=1, ascending=False)
                all_x["GROW_50_dema"]=ic.VBTGROW.run(close,distance=50, dema=True).out
                all_x["GROW_50_dema_rank"]=all_x["GROW_50_dema"].rank(axis=1, ascending=False)

                all_x["GROW_200"]=ic.VBTGROW.run(close,distance=200, ma=False).out
                all_x["GROW_200_rank"]=all_x["GROW_200"].rank(axis=1, ascending=False)
                all_x["GROW_200_ma"]=ic.VBTGROW.run(close,distance=200, ma=True).out
                all_x["GROW_200_ma_rank"]=all_x["GROW_200_ma"].rank(axis=1, ascending=False)
                all_x["GROW_200_dema"]=ic.VBTGROW.run(close,distance=200, dema=True).out
                all_x["GROW_200_dema_rank"]=all_x["GROW_200_dema"].rank(axis=1, ascending=False)
                
                all_x["OBV"]=vbt.talib("OBV").run(close,volume).real
                all_x["AD"]=vbt.talib("AD").run(high,low,close,volume).real
                
                #t=ic.VBTMA.run(close)
                #all_x["MA_ent"]=t.entries.astype(float)
                #all_x["MA_ex"]=t.exits.astype(float)
                #all_x["MA_fast_over_slow"]=t.fast_over_slow.astype(float)

                all_x["KAMA_duration"]=ic.VBTKAMATREND.run(close).duration
                #b=ic.VBTKAMATREND.run(close).duration
                all_x["KAMA_duration_rank"]=all_x["KAMA_duration"].rank(axis=1, ascending=False)
                #all_x["KAMA_duration_rank"]=b.rank(axis=1, ascending=False)

                all_x["volatility"]=ic.VBTNATR.run(high, low, close).natr

                macd=vbt.MACD.run(close, macd_wtype='simple',signal_wtype='simple')
                all_x["hist"]=macd.hist  
                all_x["macd"]=macd.macd  

                all_x["divergence"]=ic.VBTDIVERGENCE.run(close,close_ind).out
                all_x["std"]=close.rolling(5).std()
                all_x["macro_trend"]=macro.VBTMACROTREND.run(close).macro_trend
                
                t=vbt.talib("HT_TRENDMODE").run(close)
                all_x['HT_TRENDMODE']=t.integer
                
                #for nb in range(101):
                #    try:
                #        t = vbt.wqa101(nb).run(
                #        open=open_, 
                #        high=high,
                #        low=low, 
                #        close=close,
                #        volume=volume
                #        ).out
                #        all_x["WQ"+str(nb)+"_rank"]=ic.VBTRANK.run(t).rank_arr
                #    except Exception as e:
                #        pass
                #del t
            else:
                # Use x[n-1], x[n-2]... as input
                # lagged input
                for jj in range(1,sample_size+1):
                    all_x["price_past"+str(jj)]=close.shift(jj)

            cols=list(all_x.keys())
            for l in range(1,lag):
                for col in cols:
                    if "GROW" not in col: #Grow is anyhow slow, it makes no sense
                        all_x[col+"_"+str(l)]=all_x[col].shift(l)

            self.all_x[ind]=all_x          
        
        gc.collect()
        
    def defi_y(
        self,
        next_day_price:bool=True,
        distance:int=30
    ):
        self.all_y={}
        for ind in self.indexes: #CAC, DAX, NASDAQ
            all_y={}
            close=self.close_dic[ind]  
            
            if next_day_price:
                all_y["price_rise"]=100*(np.divide(close,close.shift(1))-1)
            else:    
                t=ic.VBTMINMAX.run(close,distance=distance)
                all_y['max_3mo']=t.maximum
                all_y['min_3mo']=t.minimum 
            
            self.all_y[ind]=all_y
        
    def create_empty_x_df(self, ind, s):
        return pd.MultiIndex.from_arrays([
            self.close_dic[ind].index,  #list(
            [s for ii in self.close_dic[ind].index],
            [ind for ii in self.close_dic[ind].index]
        ])
        
    def flatten(
        self, 
        input_arr,
        test_size:numbers.Number=0.2,
        ):
        
        df_total=None
        df_train=None
        df_test=None
        ts={}
        
        if "window_start" not in self.__dir__():
            self.window_start={}

        #remove the multiindex only once
        for ind in self.indexes: #CAC, DAX, NASDAQ    
            ts[ind]={}
            for col in input_arr[ind]:
                ts[ind][col]=remove_multi(input_arr[ind][col])
           
        #somehow vbt is designed with the columns in the other orders so to say, which lead to this very computer intensive function
        for ind in self.indexes: #CAC, DAX, NASDAQ
            total_len=len(self.close_dic[ind].index)
            
            learn_len=int(math.floor((1-test_size)*total_len))
            test_len=total_len-learn_len
            
            #put the window at the same place for x and y
            if ind in self.window_start:
                test_window_start=self.window_start[ind]
            else:
                test_window_start=1000 # np.random.randint(0,learn_len)
                                    #random has the advantage of the variation but we then need to reload this info if we want to test our model later, it
                                    #makes also the comparison between models more difficult
                self.window_start[ind]=test_window_start
            
            test_window_end=test_window_start+test_len
            
            learn_range=[i for i in range(0,test_window_start)]+[i for i in range(test_window_end,total_len)]
            for s in self.close_dic[ind].columns:
                dfs=[]
                for col in input_arr[ind]:
                    dfs.append(ts[ind][col][s].rename(col))
                #put columns together
                df=pd.concat(dfs,axis=1)
                #clean
                df=df.fillna(0)
                df=df.replace([np.inf, -np.inf], 0)   
                #get the index
                df.set_index(self.create_empty_x_df(ind,s) ,inplace=True)
                
                #put rows together
                df_total=pd.concat([df_total,df])
                
                #split train and test
                df_test=pd.concat([df_test,df.iloc[test_window_start:test_window_end]])
                df_train=pd.concat([df_train,df.iloc[learn_range]])         

        return df_total, df_train, df_test
    
    def unflatten(self, df, col: str=None) -> dict:
        indexes=pd.unique(df.index.get_level_values(2))
        out={}
        out2={}
        
        if col is None:
            if len(df.columns)==1:
                col=df.columns[0]
            else:
                raise ValueError("Define a column for unflatten")

        for ind in indexes:
            sub_df=df[df.index.get_level_values(2)==ind]
            out[ind]={}

            for s in pd.unique(sub_df.index.get_level_values(1)):
                sub_df2=sub_df[sub_df.index.get_level_values(1)==s]
                out[ind][s]=sub_df2[col].values
        
            out2[ind]=pd.DataFrame(data=out[ind],index=pd.unique(sub_df.index.get_level_values(0)))  
        return out2
    
    def train(
        self,
        model_name:str="model",
        model_type:str="NN"
        ):
        
        self.model_name=model_name
        
        if model_type=="NN":
            self.scaler = StandardScaler()  
            self.scaler.fit(self.x_train)
            scaled_x_train=self.scaler.transform(self.x_train)
            self.clf =  MLPRegressor(
                                #activation="tanh",
                               # solver='lbfgs',  #lbfgs
                                alpha=1e-5,  #1e-5
                                hidden_layer_sizes=(40, 4), 
                
                #activation{‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, default=’relu’
                                #random_state=1,
                                max_iter=10000)
        else:
            scaled_x_train=self.x_train
            self.clf= RandomForestRegressor(max_depth=10)
        print("starting the fitting")
        self.clf.fit(scaled_x_train, self.y_train)
        with open("models/"+model_name+".pickle", "wb") as f:
            pickle.dump(self.clf, f)
        if model_type=="NN":   
            joblib.dump(self.scaler, "models/scaler_"+self.model_name+".save") 
        print("model saved, starting the testing")
        acc=self.test()

    def train_lstm(
            self,
            model_name:str="model"
        ):
        
        trainX= np.reshape(self.x_train.values, (self.x_train.shape[0], 1, self.x_train.shape[1]))
        trainY= self.y_train.values
        
        #keras not sklearn
        self.clf = Sequential()
        self.clf.add(LSTM(4, input_shape=(1, self.x_train.shape[1])))
        self.clf.add(Dense(1))
        self.clf.compile(loss='mean_squared_error', optimizer='adam')
        self.clf.fit(trainX, trainY, epochs=1, batch_size=1) #, verbose=2
        
        with open("models/"+model_name+".pickle", "wb") as f:
            pickle.dump(self.clf, f)
        self.test_lstm()
        
    def test_lstm(self, model_name:str="model",force:bool=False):
        self.load_model(model_name,force=force)
        
        testX= np.reshape(self.x_test.values, (self.x_test.shape[0], 1, self.x_test.shape[1]))
        
        testPredict = self.clf.predict(testX)
        acc_test = np.sqrt(mean_squared_error(testPredict[:,0], self.y_test['max_3mo'].values))
        print("accurary test: "+str(acc_test))
        
    def test(self, model_name:str="model",force:bool=False):
        self.load_model(model_name,force=force)

        if self.clf.__class__ == MLPRegressor:
            scaled_x_test=self.scaler.transform(self.x_test)
            scaled_x_train=self.scaler.transform(self.x_train)
            scaled_x_df=self.scaler.transform(self.x_df)
        else:
            scaled_x_test=self.x_test
            scaled_x_train=self.x_train
            scaled_x_df=self.x_df
        acc_test = self.clf.score(scaled_x_test, self.y_test)
        print("accurary test: "+str(acc_test))
        acc_train = self.clf.score(scaled_x_train, self.y_train)
        print("accurary train: "+str(acc_train))
        acc_total = self.clf.score(scaled_x_df, self.y_df)
        print("accurary total: "+str(acc_total))

    def load_model(self, model_name:str="model",force:bool=False):
        self.model_name=model_name
        if "clf" not in self.__dir__() or force:   
            with open("models/"+model_name+".pickle", 'rb') as pickle_file:
                self.clf = pickle.load(pickle_file)
        if self.clf.__class__ == MLPRegressor and "scaler" not in self.__dir__():
            self.scaler = joblib.load("models/scaler_"+self.model_name+".save")
    
    def use(self,model_name:str, x_df,force:bool=False):
        self.load_model(model_name,force=force)

        if self.clf.__class__ == MLPRegressor:
            scaled_x_df=self.scaler.transform(x_df)
        else:
            scaled_x_df=x_df
        y=self.clf.predict(scaled_x_df)
        
        if self.clf.n_outputs_ == 1:
            return y, pd.DataFrame(data=y,columns=['price_rise'],index=x_df.index)  
        elif self.clf.n_outputs_ == 2:
            return y, pd.DataFrame(data=y,columns=['max_3mo','min_3mo'],index=x_df.index)    
    
class MLLive(ML):
    def __init__(
            self,
            period: str,
            input_ust
            ):
        
        self.indexes=["all"]
        self.ust=input_ust
        for k, v in self.ust.__dict__.items():
            if k not in self.__dict__ or getattr(self, k) is None:
                setattr(self,k,v)
                
        for key in ["close","open","low","high","data","volume"]:
            setattr(self,key+"_dic",{})
            getattr(self,key+"_dic")["all"]=getattr(self,key)
        
            setattr(self,key+"_ind_dic",{})
            getattr(self,key+"_ind_dic")["all"]=getattr(self,key+"_ind")

In [9]:
period="2007_2023_08"

In [10]:
m=ML(period)

  0%|          | 0/40 [00:00<?, ?it/s]

number of days retrieved: 4238


  0%|          | 0/32 [00:00<?, ?it/s]

number of days retrieved: 4201


  0%|          | 0/71 [00:00<?, ?it/s]

number of days retrieved: 4172


In [11]:
m.prepare(preprocessing=True,next_day_price=False,distance=10) #distance  ,lag=2
#m.prepare(data_name="all_ent_lag5")

preparation finished


In [ ]:
m.train_lstm("lstm")

In [ ]:
import matplotlib.pyplot as plt
#plt.plot(testPredict[0:500,0])
plt.plot( m.y_test['max_3mo'].values[0:500])
plt.show()

m.x_train["WQ101_rank"]

In [35]:
#m.load_model(model_name="no_pattern_forest",model_type="forest")
#m.clf.__dir__()
m.clf.feature_importances_
#m.clf.feature_names_in_
#m.clf.n_layers_
#m.clf.n_outputs_

array([0.04884579, 0.05629201, 0.        , 0.        , 0.06312029,
       0.06322592, 0.03931283, 0.01857555, 0.02652425, 0.01254287,
       0.03814744, 0.0142509 , 0.03940839, 0.01547829, 0.00092872,
       0.03435392, 0.02887069, 0.11054456, 0.04073002, 0.05055804,
       0.0527021 , 0.1009538 , 0.07191182, 0.01682592, 0.01259868,
       0.01266928, 0.03062792])

In [ ]:
d={}
t=m.clf.feature_importances_

for i, v in enumerate(m.clf.feature_names_in_):
    d[v]=t[i]
    
sorted(d.items(), key=lambda tup: tup[1], reverse=True)   
    

In [8]:
m.save("all_ent_lag5")    

In [ ]:
m.train(model_name="new_patterns_10_day") #,model_type="forest"

In [ ]:
model_name="all_ent_lag5_forest_one_day"
m.test(model_name=model_name,force=True) 

m.x_df

In [104]:
out_x=m.unflatten(m.x_df,"KAMA_duration_rank")

In [36]:
model_name="new_pattern_10_day"
a, y_df=m.use(model_name,m.x_df,force=True) 
#a, y_df=m.use(model_name,m.x_test,force=True) 
out_dic=m.unflatten(y_df,"max_3mo")
#out_dic=m.unflatten(y_df,"price_rise")
y_df

,,,max_3mo,min_3mo
Date,,,,
2007-01-02 00:00:00+01:00,AC,CAC40,2.661135,-2.038885
2007-01-03 00:00:00+01:00,AC,CAC40,2.661135,-2.038885
2007-01-04 00:00:00+01:00,AC,CAC40,2.661135,-2.038885
2007-01-05 00:00:00+01:00,AC,CAC40,2.661135,-2.038885
2007-01-08 00:00:00+01:00,AC,CAC40,2.661135,-2.038885
...,...,...,...,...
2023-07-25 00:00:00-04:00,XEL,NASDAQ,2.282533,-2.065900
2023-07-26 00:00:00-04:00,XEL,NASDAQ,2.238024,-2.032517
2023-07-27 00:00:00-04:00,XEL,NASDAQ,2.249187,-2.030407


In [48]:
scaled_x_df=m.scaler.transform(m.x_df)
m.clf.score(scaled_x_df, m.y_df)

0.1190170099620309

In [41]:
#ind="NASDAQ"
#ind="CAC40"
ind="DAX"

cand=out_dic[ind].idxmax(axis=1)
ent=ic.VBTFALSE.run(out_dic[ind]).out

for i in out_dic[ind].index:
    ent.loc[i,cand.loc[i]]=True
ex=~ent  

In [42]:
new_close=m.close_dic[ind].loc[out_dic[ind].index]#.set_index(ent.index)
new_ent=ent.set_index(new_close.index)
new_ex=ex.set_index(new_close.index)

pf=vbt.Portfolio.from_signals(new_close, new_ent,new_ex,freq="1d",
                            call_seq='auto',cash_sharing=True, fees=0.0005
                             )
pf.returns_stats()

Start                        2007-01-02 00:00:00+01:00
End                          2023-07-31 00:00:00+02:00
Period                              4201 days 00:00:00
Total Return [%]                         456128.010753
Benchmark Return [%]                        375.110787
Annualized Return [%]                        66.180394
Annualized Volatility [%]                    64.429542
Max Drawdown [%]                             81.198644
Max Drawdown Duration               1240 days 00:00:00
Sharpe Ratio                                  1.454498
Calmar Ratio                                  0.815043
Omega Ratio                                   1.251016
Sortino Ratio                                 2.361617
Skew                                          0.787733
Kurtosis                                      6.616554
Tail Ratio                                    1.188834
Common Sense Ratio                             1.97561
Value at Risk                                -0.045563
Alpha     

In [131]:
pf.stats()

Start                         2007-01-03 00:00:00-05:00
End                           2010-04-27 00:00:00-04:00
Period                                835 days 00:00:00
Start Value                                       100.0
Min Value                                      8.160048
Max Value                                    141.080202
End Value                                     22.637021
Total Return [%]                             -77.362979
Benchmark Return [%]                          37.346762
Total Time Exposure [%]                           100.0
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                              94.216022
Max Drawdown Duration                 445 days 00:00:00
Total Orders                                        361
Total Fees Paid                               11.605154
Total Trades                                        181
Win Rate [%]                                  45.555556
Best Trade [%]                               118

In [41]:
pf.plot()

/home/maxime/.local/lib/python3.8/site-packages/vectorbtpro/generic/plots_builder.py:398: UserWarning:

Subplot 'orders' does not support grouped data

/home/maxime/.local/lib/python3.8/site-packages/vectorbtpro/generic/plots_builder.py:398: UserWarning:

Subplot 'trade_pnl' does not support grouped data



FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#7f7f7f'},
              'mode': 'lines',
              'name': 'Benchmark',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'a3645d42-437f-4f46-a458-1b0c0243b844',
              'x': array([datetime.datetime(2010, 12, 8, 0, 0, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
                          datetime.datetime(2010, 12, 9, 0, 0, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
                          datetime.datetime(2010, 12, 10, 0, 0, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
                          ...,
                          datetime.datetime(2014, 3, 28, 0, 0, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
                          datetime.datetime(2014, 3, 31, 0, 0, tzinfo=<DstTzInfo 'Europe/Berlin' CEST+2:00:00 DST>),
                          datetime.datetime(2014, 4, 1, 0, 0, tzinfo=<DstTzInfo 'Europe/Berlin' CEST

In [59]:
pf.trades.records.sort_values(["entry_idx"], ascending=True).tail(20)

,id,col,size,entry_order_id,entry_idx,entry_price,entry_fees,exit_order_id,exit_idx,exit_price,exit_fees,pnl,return,direction,status,parent_id
342,95,3,0.502213,190,4213,26.899261,0.006755,191,4214,27.325291,0.006862,0.200341,0.014830,0,1,95
1034,60,12,0.078817,120,4214,173.940002,0.006855,121,4215,168.699997,0.006648,-0.426503,-0.031110,0,1,60
2317,167,27,0.347134,334,4215,38.264999,0.006642,335,4216,38.580002,0.006696,0.096010,0.007228,0,1,167
2039,47,25,0.066776,94,4216,200.358154,0.006690,95,4217,199.269257,0.006653,-0.086055,-0.006432,0,1,47
1174,65,14,0.865095,130,4217,15.366000,0.006647,131,4218,15.460000,0.006687,0.067985,0.005114,0,1,65
455,112,4,0.934990,224,4218,14.290000,0.006681,225,4220,13.520000,0.006321,-0.732943,-0.054857,0,1,112
1175,66,14,0.840998,132,4220,15.016000,0.006314,133,4221,15.094000,0.006347,0.052937,0.004192,0,1,66
456,113,4,0.898430,226,4221,14.115000,0.006341,227,4222,14.200000,0.006379,0.063647,0.005019,0,1,113
2689,156,31,0.264693,312,4222,48.150002,0.006372,313,4224,50.939999,0.006742,0.725378,0.056915,0,1,156
2148,108,26,0.006913,216,4224,1948.599976,0.006735,217,4225,1964.800049,0.006791,0.098459,0.007310,0,1,108


In [177]:
#symbols=constants.CAC40
#symbols=constants.DAX
symbols=constants.NASDAQ
#symbols=constants.IT

period=3
#index_symbol="^FCHI"
#index_symbol="^GDAXI"
index_symbol="^IXIC"

symbols=common.filter_intro_symbol(symbols,period)

In [178]:
ust=stratL.StratLIVE(symbols,str(period)+"y",index_symbol)
ust=strat.StratHold(period,input_ust=ust)

  0%|          | 0/100 [00:00<?, ?it/s]

/home/maxime/.local/lib/python3.8/site-packages/vectorbtpro/data/base.py:779: UserWarning:

Symbols have mismatching index. Dropping missing data points.



number of days retrieved: 704


In [179]:
m_live=MLLive(period, ust)

In [180]:
m_live.prepare()

file:1: RuntimeWarning:

divide by zero encountered in log

file:1: RuntimeWarning:

divide by zero encountered in divide

file:1: RuntimeWarning:

invalid value encountered in divide

file:1: RuntimeWarning:

invalid value encountered in divide

file:1: RuntimeWarning:

invalid value encountered in divide

file:1: RuntimeWarning:

invalid value encountered in divide



preparation finished


In [181]:
a, y_df_live=m_live.use("with_wq_forest",m_live.x_df)
out_dic_live=m_live.unflatten(y_df_live,"max_3mo")
y_df_live

,,,max_3mo,min_3mo
Date,,,,
2020-12-10 00:00:00-05:00,AAPL,all,7.749762,-5.476329
2020-12-11 00:00:00-05:00,AAPL,all,7.532790,-5.500365
2020-12-14 00:00:00-05:00,AAPL,all,7.628877,-5.047408
2020-12-15 00:00:00-05:00,AAPL,all,7.942358,-8.012547
2020-12-16 00:00:00-05:00,AAPL,all,10.437903,-6.836570
...,...,...,...,...
2023-09-22 00:00:00-04:00,ZS,all,9.392116,-7.395633
2023-09-25 00:00:00-04:00,ZS,all,10.245918,-6.915565
2023-09-26 00:00:00-04:00,ZS,all,10.721593,-7.126008


In [153]:
m_live.close.columns

Index(['1COV.DE', 'ADS.DE', 'AIR.DE', 'ALV.DE', 'BAS.DE', 'BAYN.DE', 'BEI.DE',
       'BMW.DE', 'BNR.DE', 'CBK.DE', 'CON.DE', 'DB1.DE', 'DBK.DE', 'DHER.DE',
       'DTE.DE', 'ENR.DE', 'FME.DE', 'FRE.DE', 'HEI.DE', 'HEN3.DE', 'HFG.DE',
       'HNR1.DE', 'IFX.DE', 'LIN.DE', 'MBG.DE', 'MRK.DE', 'MTX.DE', 'MUV2.DE',
       'PAH3.DE', 'PUM.DE', 'QIA.DE', 'RWE.DE', 'SAP.DE', 'SHL.DE', 'SIE.DE',
       'SRT.DE', 'SY1.DE', 'VNA.DE', 'VOW3.DE', 'ZAL.DE'],
      dtype='object', name='symbol')

In [182]:
ind="all"

cand=out_dic_live[ind].idxmax(axis=1)
ent=ic.VBTFALSE.run(m_live.close_dic[ind]).out

for i in m_live.close_dic[ind].index:
    ent.loc[i,cand.loc[i]]=True
ex=~ent 

In [183]:
pf_live=vbt.Portfolio.from_signals(m_live.close_dic[ind], ent,ex,freq="1d",
                            call_seq='auto',cash_sharing=True, fees=0.0005
                             )
pf_live.returns_stats()

Start                        2020-12-10 00:00:00-05:00
End                          2023-09-28 00:00:00-04:00
Period                               704 days 00:00:00
Total Return [%]                            -32.491645
Benchmark Return [%]                         12.803772
Annualized Return [%]                       -13.081274
Annualized Volatility [%]                   101.764158
Max Drawdown [%]                              76.83177
Max Drawdown Duration                479 days 00:00:00
Sharpe Ratio                                  0.283284
Calmar Ratio                                 -0.170259
Omega Ratio                                   1.045934
Sortino Ratio                                 0.465058
Skew                                          1.997075
Kurtosis                                     19.086779
Tail Ratio                                     1.07427
Common Sense Ratio                            0.933742
Value at Risk                                -0.069423
Alpha     

In [184]:
pf_live.plot()

/home/maxime/.local/lib/python3.8/site-packages/vectorbtpro/generic/plots_builder.py:398: UserWarning:

Subplot 'orders' does not support grouped data

/home/maxime/.local/lib/python3.8/site-packages/vectorbtpro/generic/plots_builder.py:398: UserWarning:

Subplot 'trade_pnl' does not support grouped data



FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#7f7f7f'},
              'mode': 'lines',
              'name': 'Benchmark',
              'showlegend': True,
              'type': 'scatter',
              'uid': '87388583-c47f-4747-a168-24fb2e119fa9',
              'x': array([datetime.datetime(2020, 12, 10, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2020, 12, 11, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2020, 12, 14, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2023, 9, 26, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2023, 9, 27, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(

In [186]:
pf.plot()

/home/maxime/.local/lib/python3.8/site-packages/vectorbtpro/generic/plots_builder.py:398: UserWarning:

Subplot 'orders' does not support grouped data

/home/maxime/.local/lib/python3.8/site-packages/vectorbtpro/generic/plots_builder.py:398: UserWarning:

Subplot 'trade_pnl' does not support grouped data



FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#7f7f7f'},
              'mode': 'lines',
              'name': 'Benchmark',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'ad5a6039-949a-4dc7-9a89-968d39e887d9',
              'x': array([datetime.datetime(2007, 1, 3, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2007, 1, 4, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2007, 1, 5, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2023, 7, 27, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2023, 7, 28, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2023, 

In [191]:
pf.trades.records.sort_values(["entry_idx"], ascending=True).tail(50)

,id,col,size,entry_order_id,entry_idx,entry_price,entry_fees,exit_order_id,exit_idx,exit_price,exit_fees,pnl,return,direction,status,parent_id
318,41,14,2.917258e+05,82,3984,78.540001,11456.072030,83,3997,94.750000,13820.509450,4.703598e+06,0.205288,0,1,41
704,85,48,5.596553e+05,170,3997,49.340000,13806.695847,171,3998,47.029999,13160.293612,-1.319771e+06,-0.047795,0,1,85
319,42,14,2.640518e+05,84,3998,99.580002,13147.139896,85,4009,116.919998,15436.468611,4.550074e+06,0.173044,0,1,42
921,39,51,4.505935e+05,78,4009,68.447685,15421.039856,79,4014,68.774948,15494.771114,1.165467e+05,0.003779,0,1,39
320,43,14,2.655109e+05,86,4014,116.599998,15479.284086,87,4016,119.220001,15827.103703,6.643328e+05,0.021459,0,1,43
614,22,46,7.445100e+05,44,4016,42.474339,15811.284509,45,4017,39.855167,14836.284979,-1.980647e+06,-0.062634,0,1,22
321,44,14,2.607116e+05,88,4017,113.699997,14821.456108,89,4018,112.589996,14676.761071,-3.188883e+05,-0.010758,0,1,44
369,18,25,2.883862e+06,36,4018,10.168372,14662.091644,37,4019,10.197453,14704.024939,5.450047e+04,0.001859,0,1,18
63,42,6,1.478097e+05,84,4019,198.759995,14689.328262,85,4027,210.899994,15586.533137,1.764134e+06,0.060048,0,1,42
982,60,52,2.131547e+05,120,4027,146.100067,15570.954394,121,4031,148.549393,15831.996961,4.906822e+05,0.015756,0,1,60
